<a href="https://colab.research.google.com/github/jun-wei-lin/NCHU/blob/main/AIoT-DA/HW4%3DAutoML%20and%20Ensemble/Hw4_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**步驟 1: 安裝相關套件**<br>
在 Google Colab 中安裝所需的庫：

In [1]:
!pip install pycaret
!pip install optuna
!pip install h2o
!pip install tpot

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.8/265.8 MB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for h2o: filename=h2o-3.46.0.6-py2.py3-none-any.whl size=265859786 sha256=72a6726a37142c5ca0bb399b5755e545fa85fbd42451d77e582047272206f66b
  Stored in directory: /root/.cache/pip/wheels/0c/95/eb/b449c73f478dbc4557b80170cf7f2aa6db15862877e9a63536
Successfully built h2o
  Using cached TPOT-0.12.2-py3-none-any.whl.metadata (2.0 kB)
  Using cached deap-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (13 kB)
  Using cached update_checker-0.18.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached stopit-1.1.2.tar.gz (18 kB)
  Preparing metadata (setup.py) ... done
Using cached TPOT-0.12.2-py3-none-any.whl (87 kB)
Using cached deap-1.4.1-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (135 kB)
Using cached update_checker-0.18.0-py3-none-any.whl 

**步驟 2: 加載與預處理數據**<br>
使用 Seaborn 提供的 Titanic 數據集並進行預處理：

In [2]:
import seaborn as sns
import pandas as pd

# 加載 Titanic 數據集
df = sns.load_dataset('titanic')

# 刪除與預測無關的欄位
df = df.drop(['deck', 'embarked', 'alive', 'class'], axis=1)

# 填補缺失值
df['age'] = df['age'].fillna(df['age'].median())
df['embark_town'] = df['embark_town'].fillna(df['embark_town'].mode()[0])

# 編碼類別變量
df = pd.get_dummies(df, drop_first=True)


**步驟 3: 使用 PyCaret 進行自動化處理**<br>
**3.1**設置 PyCaret 環境並自動特徵工程<br>
使用 PyCaret 的 setup 進行特徵工程：

In [3]:
from pycaret.classification import setup, compare_models, tune_model

# 設置 PyCaret 環境
clf1 = setup(
    data=df,
    target='survived',
    session_id=123,              # 固定隨機種子
    normalize=True,              # 標準化
    feature_selection=True,      # 啟用特徵選擇
    remove_multicollinearity=True,  # 去除多重共線性
    pca=True                     # 啟用 PCA 降維（可選）
)


[LightGBM] [Info] Number of positive: 239, number of negative: 384
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000421 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2079
[LightGBM] [Info] Number of data points in the train set: 623, number of used features: 10
[LightGBM] [Info] [binary:BoostFromScore]: pavg=0.383628 -> initscore=-0.474179
[LightGBM] [Info] Start training from score -0.474179
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, bes

,Description,Value
0,Session id,123
1,Target,survived
2,Target type,Binary
3,Original data shape,"(891, 13)"
4,Transformed data shape,"(891, 3)"
5,Transformed train set shape,"(623, 3)"
6,Transformed test set shape,"(268, 3)"
7,Numeric features,5
8,Preprocess,True
9,Imputation type,simple


**3.2** 比較多個模型並選擇最佳模型<br>
使用 PyCaret 自動比較模型性能：

In [4]:
# 比較所有模型並選擇最佳模型
best_models = compare_models(n_select=3, sort='Accuracy')
print(best_models)  # 顯示性能最好的 3 個模型


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7029,0.7472,0.4853,0.6605,0.5532,0.3396,0.3527,0.3440
ada,Ada Boost Classifier,0.6980,0.7054,0.4649,0.6609,0.5305,0.3231,0.3417,0.5680
knn,K Neighbors Classifier,0.6978,0.7208,0.5600,0.6170,0.5831,0.3480,0.3518,0.1570
et,Extra Trees Classifier,0.6917,0.7348,0.5813,0.6109,0.5931,0.3456,0.3480,0.2750
rf,Random Forest Classifier,0.6886,0.7354,0.5855,0.6076,0.5916,0.3409,0.3448,0.2640
lightgbm,Light Gradient Boosting Machine,0.6836,0.7217,0.5645,0.6014,0.5768,0.3257,0.3303,0.4360
lda,Linear Discriminant Analysis,0.6820,0.6212,0.2417,0.6424,0.3256,0.2176,0.2694,0.1760
lr,Logistic Regression,0.6804,0.6238,0.2375,0.6401,0.3151,0.2116,0.2613,0.9120
ridge,Ridge Classifier,0.6788,0.6215,0.2333,0.6390,0.3125,0.2079,0.2576,0.0970
dt,Decision Tree Classifier,0.6774,0.6513,0.5980,0.5791,0.5867,0.3227,0.3243,0.0980


Processing:   0%|          | 0/67 [00:00<?, ?it/s]

[GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False), AdaBoostClassifier(algorithm='SAMME.R', estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=123), KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=-1, n_neighbors=5, p=2,
                     weights='uniform')]


**3.3** 微調模型的超參數<br>
對最佳模型進行超參數優化：

In [5]:
# 微調第一個模型
tuned_model = tune_model(best_models[0], optimize='AUC')
print(tuned_model)  # 顯示優化後的模型


,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
Fold,,,,,,,
0,0.6190,0.7601,0.0000,0.0000,0.0000,0.0000,0.0000
1,0.6190,0.7083,0.0000,0.0000,0.0000,0.0000,0.0000
2,0.6190,0.8729,0.0000,0.0000,0.0000,0.0000,0.0000
3,0.6290,0.7246,0.0000,0.0000,0.0000,0.0000,0.0000
4,0.6129,0.6288,0.0000,0.0000,0.0000,0.0000,0.0000
5,0.6129,0.7939,0.0000,0.0000,0.0000,0.0000,0.0000
6,0.6129,0.7610,0.0000,0.0000,0.0000,0.0000,0.0000
7,0.6129,0.7862,0.0000,0.0000,0.0000,0.0000,0.0000
8,0.6129,0.6798,0.0000,0.0000,0.0000,0.0000,0.0000


Processing:   0%|          | 0/7 [00:00<?, ?it/s]

Fitting 10 folds for each of 10 candidates, totalling 100 fits


Original model was better than the tuned model, hence it will be returned. NOTE: The display metrics are for the tuned model (not the original one).
GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=123, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


**步驟 4: 使用 Optuna 進行模型選擇和超參數調優**<br>
Optuna 提供更靈活的模型選擇和調優方法：

**4.1** 使用 Optuna 調整模型參數

In [6]:
import optuna
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score

# 切分數據集
X = df.drop(columns='survived')
y = df['survived']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 定義目標函數
def objective(trial):
    model_name = trial.suggest_categorical('model', ['RandomForest', 'GradientBoosting'])

    if model_name == 'RandomForest':
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        max_depth = trial.suggest_int('max_depth', 2, 32, log=True)
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
    else:
        learning_rate = trial.suggest_float('learning_rate', 0.01, 0.3)
        n_estimators = trial.suggest_int('n_estimators', 10, 200)
        model = GradientBoostingClassifier(learning_rate=learning_rate, n_estimators=n_estimators, random_state=42)

    # 計算交叉驗證分數
    scores = cross_val_score(model, X_train, y_train, cv=3, scoring='accuracy')
    return scores.mean()

# 開始調參
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=50)

# 查看最佳參數和模型
print("Best trial:", study.best_trial)


Best trial: FrozenTrial(number=13, state=TrialState.COMPLETE, values=[0.8356794194471039], datetime_start=datetime.datetime(2024, 12, 5, 0, 26, 2, 271091), datetime_complete=datetime.datetime(2024, 12, 5, 0, 26, 2, 854033), params={'model': 'RandomForest', 'n_estimators': 130, 'max_depth': 8}, user_attrs={}, system_attrs={}, intermediate_values={}, distributions={'model': CategoricalDistribution(choices=('RandomForest', 'GradientBoosting')), 'n_estimators': IntDistribution(high=200, log=False, low=10, step=1), 'max_depth': IntDistribution(high=32, log=True, low=2, step=1)}, trial_id=13, value=None)


**步驟 5: 使用 AutoML 工具進行自動化處理**

**5.1** H2O AutoML

In [7]:
import h2o
from h2o.automl import H2OAutoML

# 啟動 H2O
h2o.init()

# 加載數據
df_h2o = h2o.H2OFrame(df)
df_h2o['survived'] = df_h2o['survived'].asfactor()

# 自動建模（快速運行）
aml = H2OAutoML(
    max_runtime_secs=120,        # 限制時間為 2 分鐘
    nfolds=3,                    # 減少交叉驗證次數
    include_algos=["GBM", "XGBoost"],  # 僅使用快速算法
    seed=42
)
aml.train(y='survived', training_frame=df_h2o)

# 查看最佳模型
lb = aml.leaderboard
print(lb.head())


Checking whether there is an H2O instance running at http://localhost:54321..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.25" 2024-10-15; OpenJDK Runtime Environment (build 11.0.25+9-post-Ubuntu-1ubuntu122.04); OpenJDK 64-Bit Server VM (build 11.0.25+9-post-Ubuntu-1ubuntu122.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.10/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmp_m_3ly7j
  JVM stdout: /tmp/tmp_m_3ly7j/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmp_m_3ly7j/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,03 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.46.0.6
H2O_cluster_version_age:,1 month and 2 days
H2O_cluster_name:,H2O_from_python_unknownUser_hc5fkp
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.170 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
AutoML progress: |███████████████████████████████████████████████████████████████| (done) 100%
model_id                                          auc    logloss     aucpr    mean_per_class_error      rmse       mse
GBM_4_AutoML_1_20241205_02638                0.878242   0.407498  0.8525                  0.182101  0.354457  0.12564
GBM_2_AutoML_1_20241205_02638                0.876788   0.410745  0.850612                0.18584   0.356721  0.12725
GBM_grid_1_AutoML_1_20241205_02638_model_32  0.876615   0.41066   0.84279                 0.170813  0.355664  0.126497
GBM_5_AutoML_1_20241205_02638                0.876216   0.407108  0.846462                0.171005  0.35319   0.124743
GBM_grid_1_AutoML_1_20241205_02638_model_33  0.87584    0.410666  0.852655                0.181934  0.356323  0.126966
GBM_grid_1_AutoML_1_20241205_02638_model_11  0.875776   0.412213  0.848418                0.182125

**5.2** TPOT

In [8]:
from tpot import TPOTClassifier

tpot = TPOTClassifier(
    generations=5,      # 5 代
    population_size=20,    # 每代 20 個模型
    max_time_mins=30,     # 最多運行 30 分鐘
    config_dict='TPOT light', # 使用輕量化配置
    verbosity=2,
    random_state=42
)

tpot.fit(X_train, y_train)
print(tpot.fitted_pipeline_)
print("Test Accuracy:", tpot.score(X_test, y_test))


Optimization Progress:   0%|          | 0/20 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8230178272431793

Generation 2 - Current best internal CV score: 0.8230178272431793

Generation 3 - Current best internal CV score: 0.8244262779474048

Generation 4 - Current best internal CV score: 0.8244262779474048

Generation 5 - Current best internal CV score: 0.8244262779474048

Best pipeline: DecisionTreeClassifier(input_matrix, criterion=entropy, max_depth=10, min_samples_leaf=20, min_samples_split=12)
Pipeline(memory=None,
         steps=[('decisiontreeclassifier',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='entropy', max_depth=10,
                                        max_features=None, max_leaf_nodes=None,
                                        min_impurity_decrease=0.0,
                                        min_samples_leaf=20,
                                        min_samples_split=12,
                                        min_weight_f